In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table align="left">

  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/structured_data/rapid_prototyping_bqml_automl.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/structured_data/rapid_prototyping_bqml_automl.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
</table>

# BQML and AutoML - Experimenting with Vertex AI

<img src="https://storage.googleapis.com/rafacarv-public-bucket-do-not-delete/abalone/rapid_pipeline.png" />

## Before you begin

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

1. If you are running this notebook locally, you will need to install the [Cloud SDK](https://cloud.google.com/sdk).

1. You'll use the *gcloud* command throughout this notebook. In the following cell, enter your project name and run the cell to authenticate yourself with the Google Cloud and initialize your *gcloud* configuration settings.

**For this lab, we're going to use region us-central1 for all our resources (BigQuery training data, Cloud Storage bucket, model and endpoint locations, etc.). Those resources can be deployed in other regions, as long as they're consistently co-located, but we're going to use one fixed region to keep things as simple and error free as possible.**

In [ ]:
import os

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# Google Cloud Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_GOOGLE_CLOUD_NOTEBOOK:
    USER_FLAG = "--user"

### Authenticate with GCP

In [ ]:
# Install Python package dependencies.
print("Installing libraries")
! pip3 install {USER_FLAG} --quiet --upgrade google-cloud-pipeline-components==0.1.7 kfp==1.8.2
! pip3 install {USER_FLAG} --quiet --upgrade google-cloud-aiplatform==1.4.3 google-cloud-bigquery

In [ ]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

### Determine some project and pipeline variables

### Required imports

In [ ]:
import sys
from datetime import datetime
from typing import NamedTuple

from google.cloud import aiplatform as vertex
from google_cloud_pipeline_components import \
    aiplatform as vertex_pipeline_components
from kfp import dsl
from kfp.v2 import compiler
from kfp.v2.dsl import (Artifact, Dataset, Input, Metrics, Model, Output,
                        component)

### Required Variables

Make sure the GCS bucket and the BigQuery Dataset do not exist. This script may **delete** any existing content.

Your bucket must be on the same region as your Vertex AI resources.

BQ region can be US or EU

In [ ]:
PROJECT_ID = "your-project-id"  # @param {type:"string"}
REGION = "us-central1"  # @param {type:"string"}
BUCKET_NAME = "j90wipxexhrgq3cquanc5"  # @param {type:"string"}
PIPELINE_JSON_PKG_PATH = "rapid_prototyping.json"
PIPELINE_ROOT = f"gs://{BUCKET_NAME}/pipeline_root"
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
DATA_FOLDER = f"{BUCKET_NAME}/data"

TEST_DATASET_CSV_LOCATION = f"gs://{DATA_FOLDER}/test_dataset"
RAW_INPUT_DATA = f"gs://{DATA_FOLDER}/abalone.csv"
BQ_DATASET = "j90wipxexhrgq3cquanc5"  # @param {type:"string"}
BQ_LOCATION = "EU"  # @param {type:"string"}
BQ_LOCATION = BQ_LOCATION.upper()
BQML_EXPORT_LOCATION = f"gs://{BUCKET_NAME}/artifacts/bqml/"
GCS_BATCH_PREDICTION_OUTPUT_PREFIX = f"gs://{BUCKET_NAME}/predictions/"

image_prefix = REGION.split("-")[0]
BQML_SERVING_CONTAINER_IMAGE_URI = (
    f"{image_prefix}-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-6:latest"
)

In [ ]:
import os

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# If on Google Cloud Notebooks, then don't execute this code
if not IS_GOOGLE_CLOUD_NOTEBOOK:
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

In [ ]:
if os.getenv("IS_TESTING"):
    !gcloud --quiet components install beta
    !gcloud --quiet components update
!gcloud config set project $PROJECT_ID
!gcloud config set ai/region $REGION

## The Abalone Dataset

<img src="https://storage.googleapis.com/rafacarv-public-bucket-do-not-delete/abalone/dataset.png" />

<p>Dataset Credits</p>
<p>Dua, D. and Graff, C. (2019). UCI Machine Learning Repository <a href="http://archive.ics.uci.edu/ml">http://archive.ics.uci.edu/ml</a>. Irvine, CA: University of California, School of Information and Computer Science.</p>

<p><a href="https://archive.ics.uci.edu/ml/datasets/abalone">Direct link<a/></p>
    
    
#### Attribute Information:

<p>Given is the attribute name, attribute type, the measurement unit and a brief description. The number of rings is the value to predict: either as a continuous value or as a classification problem.</p>

<body>
	<table>
		<tr>
			<th>Name</th>
			<th>Data Type</th>
			<th>Measurement Unit</th>
			<th>Description</th>
		</tr>
		<tr>
			<td>Sex</td>
            <td>nominal</td>
            <td>--</td>
            <td>M, F, and I (infant)</td>
		</tr>
		<tr>
			<td>Length</td>
            <td>continuous</td>
            <td>mm</td>
            <td>Longest shell measurement</td>
		</tr>
		<tr>
			<td>Diameter</td>
            <td>continuous</td>
            <td>mm</td>
            <td>perpendicular to length</td>
		</tr>
		<tr>
			<td>Height</td>
            <td>continuous</td>
            <td>mm</td>
            <td>with meat in shell</td>
		</tr>
		<tr>
			<td>Whole weight</td>
            <td>continuous</td>
            <td>grams</td>
            <td>whole abalone</td>
		</tr>
		<tr>
			<td>Shucked weight</td>
            <td>continuous</td>
            <td>grams</td>
            <td>weight of meat</td>
		</tr>
		<tr>
			<td>Viscera weight</td>
            <td>continuous</td>
            <td>grams</td>
            <td>gut weight (after bleeding)</td>
		</tr>
		<tr>
			<td>Shell weight</td>
            <td>continuous</td>
            <td>grams</td>
            <td>after being dried</td>
		</tr>
        <tr>
			<td>Rings</td>
            <td>integer</td>
			<td>--</td>
            <td>+1.5 gives the age in years</td>
		</tr>
	</table>
</body>


### Downloading the data

If the bucket does not exist, the script below will create it.

After creating the bucket, the cell below will download the dataset into a CSV file and save it in GCS

In [ ]:
! gsutil ls -b gs://{BUCKET_NAME} || gsutil mb -l {REGION} gs://{BUCKET_NAME}

! gsutil cp gs://cloud-samples-data/vertex-ai/community-content/datasets/abalone/abalone.data {RAW_INPUT_DATA}

## Pipeline Components

### Import to BQ

This component takes the csv file and imports it to a table in BigQuery. If the dataset does not exist, it will be created. If a table with the same name already exists, it will be deleted and recreated

In [ ]:
@component(base_image="python:3.9", packages_to_install=["google-cloud-bigquery"])
def import_data_to_bigquery(
    project: str,
    bq_location: str,
    bq_dataset: str,
    gcs_data_uri: str,
    raw_dataset: Output[Artifact],
    table_name_prefix: str = "abalone",
):
    from google.cloud import bigquery

    # Construct a BigQuery client object.
    client = bigquery.Client(project=project, location=bq_location)

    def load_dataset(gcs_uri, table_id):
        job_config = bigquery.LoadJobConfig(
            schema=[
                bigquery.SchemaField("Sex", "STRING"),
                bigquery.SchemaField("Length", "NUMERIC"),
                bigquery.SchemaField("Diameter", "NUMERIC"),
                bigquery.SchemaField("Height", "NUMERIC"),
                bigquery.SchemaField("Whole_weight", "NUMERIC"),
                bigquery.SchemaField("Shucked_weight", "NUMERIC"),
                bigquery.SchemaField("Viscera_weight", "NUMERIC"),
                bigquery.SchemaField("Shell_weight", "NUMERIC"),
                bigquery.SchemaField("Rings", "NUMERIC"),
            ],
            skip_leading_rows=1,
            # The source format defaults to CSV, so the line below is optional.
            source_format=bigquery.SourceFormat.CSV,
        )
        print(f"Loading {gcs_uri} into {table_id}")
        load_job = client.load_table_from_uri(
            gcs_uri, table_id, job_config=job_config
        )  # Make an API request.

        load_job.result()  # Waits for the job to complete.
        destination_table = client.get_table(table_id)  # Make an API request.
        print("Loaded {} rows.".format(destination_table.num_rows))

    def create_dataset_if_not_exist(bq_dataset_id, bq_location):
        print(
            "Checking for existence of bq dataset. If it does not exist, it creates one"
        )
        dataset = bigquery.Dataset(bq_dataset_id)
        dataset.location = bq_location
        dataset = client.create_dataset(dataset, exists_ok=True, timeout=300)
        print(f"Created dataset {dataset.full_dataset_id} @ {dataset.location}")

    bq_dataset_id = f"{project}.{bq_dataset}"
    create_dataset_if_not_exist(bq_dataset_id, bq_location)

    raw_table_name = f"{table_name_prefix}_raw"
    table_id = f"{project}.{bq_dataset}.{raw_table_name}"
    print("Deleting any tables that might have the same name on the dataset")
    client.delete_table(table_id, not_found_ok=True)
    print("will load data to table")
    load_dataset(gcs_data_uri, table_id)

    raw_dataset_uri = f"bq://{table_id}"
    raw_dataset.uri = raw_dataset_uri

### Split Datasets

Splits the dataset in 3 slices:
- TRAIN
- EVALUTE
- TEST


AutoML and BigQuery ML use different nomenclatures for data splits, so 2 BQ tables will be created.

#### BQML
- How BQML splits the data: [link](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-hyperparameter-tuning#data_split)

#### AutoML
How AutoML splits the data: [link](https://cloud.google.com/vertex-ai/docs/general/ml-use?hl=da&skip_cache=false)

<ul>
    <li>Model trials
<p>The training set is used to train models with different preprocessing, architecture, and hyperparameter option combinations. These models are evaluated on the validation set for quality, which guides the exploration of additional option combinations. The best parameters and architectures determined in the parallel tuning phase are used to train two ensemble models as described below.</p></li>

<li>Model evaluation
<p>
Vertex AI trains an evaluation model, using the training and validation sets as training data. Vertex AI generates the final model evaluation metrics on this model, using the test set. This is the first time in the process that the test set is used. This approach ensures that the final evaluation metrics are an unbiased reflection of how well the final trained model will perform in production.</p></li>

<li>Serving model
<p>A model is trained with the training, validation, and test sets, to maximize the amount of training data. This model is the one that you use to request predictions.</p></li>


In [ ]:
@component(
    base_image="python:3.9",
    packages_to_install=[
        "google-cloud-bigquery",
        "pandas",
        "pyarrow",
        "fsspec",
        "gcsfs",
    ],
)  # pandas, pyarrow and fsspec required to export bq data to csv
def split_datasets(
    raw_dataset: Input[Artifact],
    bqml_dataset: Output[Dataset],
    test_dataset_folder: str,
    bq_location: str,
) -> NamedTuple(
    "bqml_split",
    [
        ("dataset_uri", str),
        ("test_features_jsonl_filenames", list),
        ("test_labels_csv_filename", str),
        ("train_dataset_uri", str),
        ("test_dataset_uri", str),
        ("validate_dataset_uri", str),
    ],
):

    from collections import namedtuple

    import gcsfs
    from google.cloud import bigquery

    raw_dataset_uri = raw_dataset.uri
    table_name = raw_dataset_uri.split("bq://")[-1]
    print(table_name)
    raw_dataset_uri = table_name.split(".")
    print(raw_dataset_uri)
    project = raw_dataset_uri[0]
    bq_dataset = raw_dataset_uri[1]
    bq_raw_table = raw_dataset_uri[2]

    client = bigquery.Client(project=project, location=bq_location)

    def split_dataset(table_name_dataset):
        training_dataset_table_name = f"{project}.{bq_dataset}.{table_name_dataset}"
        split_query = f"""
        CREATE OR REPLACE TABLE
            `{training_dataset_table_name}`
           AS
        SELECT
          Sex,
          Length,
          Diameter,
          Height,
          Whole_weight,
          Shucked_weight,
          Viscera_weight,
          Shell_weight,
          Rings,
            CASE(ABS(MOD(FARM_FINGERPRINT(TO_JSON_STRING(f)), 10)))
              WHEN 9 THEN 'TEST'
              WHEN 8 THEN 'VALIDATE'
              ELSE 'TRAIN' END AS split_col
        FROM
          `{project}.{bq_dataset}.abalone_raw` f
        """
        dataset_uri = f"{project}.{bq_dataset}.{bq_raw_table}"
        print("Splitting the dataset")
        query_job = client.query(split_query)  # Make an API request.
        query_job.result()
        print(dataset_uri)
        print(split_query.replace("\n", " "))
        return training_dataset_table_name

    def create_separate_tables(splits, training_dataset_table_name):
        output = {}
        for s in splits:
            destination_table_name = f"abalone_{s}"
            query = f"""
             CREATE OR REPLACE TABLE `{project}.{bq_dataset}.{destination_table_name}` AS SELECT
          Sex,
          Length,
          Diameter,
          Height,
          Whole_weight,
          Shucked_weight,
          Viscera_weight,
          Shell_weight,
          Rings 
          FROM `{training_dataset_table_name}`  f
          WHERE 
          f.split_col = '{s}'
          """
            print(f"Creating table for {s} --> {destination_table_name}")
            print(query.replace("\n", " "))
            output[s] = destination_table_name
            query_job = client.query(query)  # Make an API request.
            query_job.result()

        print(output)
        return output

    def create_bqml_dataset(training_dataset_table_name):
        bqml_dataset_table_name = "dataset_bqml"

        query = f"""
        CREATE OR REPLACE TABLE
          `{project}.{bq_dataset}.{bqml_dataset_table_name}`  AS
        SELECT
          Sex,
          Length,
          Diameter,
          Height,
          Whole_weight,
          Shucked_weight,
          Viscera_weight,
          Shell_weight,
          Rings,
          CASE(split_col)
            WHEN 'VALIDATE' THEN 'EVAL'
            WHEN 'TRAIN' THEN 'TRAIN'
            WHEN 'TEST' THEN 'TEST'
        END
          AS split_col
        FROM
          `{project}.{bq_dataset}.{training_dataset_table_name}`  f
        WHERE
          split_col IN ('VALIDATE',
            'TRAIN')
        """

        # print(query)
        query_job = client.query(query)  # Make an API request.
        query_job.result()
        return bqml_dataset_table_name

    def export_test_features_to_gcs(bq_test_table_name, gcs_export_path_prefix):
        query_string = f"""
        SELECT
          Sex,
          Length,
          Diameter,
          Height,
          Whole_weight,
          Shucked_weight,
          Viscera_weight,
          Shell_weight,
          Rings
        FROM `{project}.{bq_dataset}.{bq_test_table_name}`  f
        """
        print(f"Exporting test dataset {project}.{bq_dataset}.{bq_test_table_name}")
        print(query_string.replace("\n", " "))
        dataframe = (
            client.query(query_string)
            .result()
            .to_dataframe(
                # Optionally, explicitly request to use the BigQuery Storage API. As of
                # google-cloud-bigquery version 1.26.0 and above, the BigQuery Storage
                # API is used by default.
                create_bqstorage_client=True,
            )
        )

        test_labels_csv_filename = f"{gcs_export_path_prefix}/test_labels.csv"
        labels = dataframe["Rings"]
        print(f"Exporting test labels into {test_labels_csv_filename}")
        labels.to_csv(test_labels_csv_filename, index=False, header=True)

        test_features_jsonl_filename = f"{gcs_export_path_prefix}/test_features.jsonl"
        features = dataframe.drop(columns=["Rings"])
        jsonl = features.to_json(orient="records", lines=True)
        gcs = gcsfs.GCSFileSystem()
        with gcs.open(test_features_jsonl_filename, "w") as text_file:
            text_file.write(jsonl)

        print(f"Exporting test labels into {test_features_jsonl_filename}")
        return test_features_jsonl_filename, test_labels_csv_filename

    table_name_dataset = "dataset"

    dataset_uri = split_dataset(table_name_dataset)
    splits = ["TRAIN", "VALIDATE", "TEST"]
    table_names_dict = create_separate_tables(splits, dataset_uri)
    bqml_dataset_table_name = create_bqml_dataset(table_name_dataset)
    test_table_name = table_names_dict["TEST"]
    (
        test_features_jsonl_filename,
        test_labels_csv_filename,
    ) = export_test_features_to_gcs(test_table_name, test_dataset_folder)
    dataset_uri = "bq://" + dataset_uri
    train_dataset_uri = f"bq://{project}.{bq_dataset}.{table_names_dict['TRAIN']}"
    test_dataset_uri = f"bq://{project}.{bq_dataset}.{table_names_dict['TEST']}"
    validate_dataset_uri = f"bq://{project}.{bq_dataset}.{table_names_dict['VALIDATE']}"
    bqml_dataset_uri = f"bq://{project}.{bq_dataset}.{bqml_dataset_table_name}"

    print(f"dataset: {dataset_uri}")
    print(f"training: {train_dataset_uri}")
    print(f"test: {test_dataset_uri}")
    print(f"validation: {validate_dataset_uri}")

    # dataset.uri = dataset_uri
    # train_dataset.uri = train_dataset_uri
    # test_dataset.uri = test_dataset_uri
    # validate_dataset.uri = validate_dataset_uri
    bqml_dataset.uri = bqml_dataset_uri

    result_tuple = namedtuple(
        "bqml_split",
        [
            "dataset_uri",
            "test_features_jsonl_filenames",
            "test_labels_csv_filename",
            "train_dataset_uri",
            "test_dataset_uri",
            "validate_dataset_uri",
        ],
    )
    return result_tuple(
        dataset_uri=str(dataset_uri),
        test_features_jsonl_filenames=[test_features_jsonl_filename],
        test_labels_csv_filename=test_labels_csv_filename,
        train_dataset_uri=train_dataset_uri,
        test_dataset_uri=test_dataset_uri,
        validate_dataset_uri=validate_dataset_uri,
    )

### Train BQML Model

On this demo, we will use a simple linear regression model on BQML. However, you can be creative with other model architectures, such as Deep Neural Networks, XGboost, Logistic Regression, etc.

For a full list of models supported by BQML, look here: [End-to-end user journey for each model](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-e2e-journey)

BQML also supports **Hyperparameter Tuning**. On this demo, we are doing 4 different trials. You can expand this following the documentation: [BigQuery ML Hyperparameter Tuning Overview](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-hp-tuning-overview)


In [ ]:
@component(base_image="python:3.9", packages_to_install=["google-cloud-bigquery"])
def train_bqml_model(
    dataset: Input[Dataset],
    bqml_model: Output[Artifact],
    bq_location: str,
    model_name: str = "linear_regression_model",
    num_trials: int = 4,
) -> NamedTuple("bqml_training", [("query", str)]):
    from collections import namedtuple

    from google.cloud import bigquery

    dataset_uri = dataset.uri
    table_name = dataset_uri.split("bq://")[-1]
    print(table_name)
    uri_parts = table_name.split(".")
    print(uri_parts)
    project = uri_parts[0]
    bq_dataset = uri_parts[1]

    client = bigquery.Client(project=project, location=bq_location)

    model_table_name = f"{project}.{bq_dataset}.{model_name}"

    model_options = """OPTIONS
      ( MODEL_TYPE='LINEAR_REG',
        LS_INIT_LEARN_RATE=0.15,
        L1_REG=1,
        MAX_ITERATIONS=5,
        DATA_SPLIT_COL='split_col',
        DATA_SPLIT_METHOD='CUSTOM',
        input_label_cols=['Rings']
        
    """
    if num_trials > 0:
        model_options += f""", 
        
        NUM_TRIALS={num_trials},
        HPARAM_TUNING_OBJECTIVES=['mean_squared_error']"""
    model_options += ")" ""

    query = f"""
    CREATE OR REPLACE MODEL
      `{model_table_name}`
      {model_options}
     AS
    SELECT
      Sex,
      Length,
      Diameter,
      Height,
      Whole_weight,
      Shucked_weight,
      Viscera_weight,
      Shell_weight,
      Rings,
      split_col
    FROM
      `{table_name}`;
    """

    print(query.replace("\n", " "))
    query_job = client.query(query)  # Make an API request.
    print(query_job.job_id)
    query_job.result()
    bqml_model.uri = f"bq://{model_table_name}"

    result_tuple = namedtuple("bqml_training", ["query"])

    return result_tuple(query=str(query))

### Export BQML Model

Once the BQML model is trained, you will need to export it to a GCS bucket, so it can later be imported to Vertex AI.

For more information on exporting your model, please follow the [documentation].(https://cloud.google.com/bigquery-ml/docs/exporting-models)

The doc above also talks about limitations on the model export statement. 

As of September 2021, an important one to be aware is the following:

- The model **doesn't contain** the information about the **feature engineering** performed in the `SELECT` statement during training, so you would need to manually convert the input data before feeding into the exported model.


In [ ]:
@component(base_image="python:3.9", packages_to_install=["google-cloud-bigquery"])
def export_bqml_to_tf(
    project: str,
    export_location: str,
    bqml_model: Input[Model],
    tf_model: Output[Artifact],
    bq_location: str,
):
    from google.cloud import bigquery

    bqml_table_name = bqml_model.uri.split("/")[-1]
    query = f"""
     EXPORT MODEL `{bqml_table_name}`
    OPTIONS(URI = '{export_location}')

    """
    client = bigquery.Client(project=project, location=bq_location)
    query_job = client.query(query)
    query_job.result()

    tf_model.uri = export_location

### Model Evaluation

After your model has been trained, it is now time to run some predictions on data that it never saw before. In order to do that, we will make a batch prediction request to the recently trained model. In the previous data split, we reserved some instances on the **TEST** dataset to be used for this

Once the batch prediction job is finalized, we will compare the predictions to the actual results, that were saved in the file that we are calling ground truth.

In this demo, since this is a regression model, we are going to use RMSE to compare the results.


In [ ]:
@component(
    base_image="python:3.9",
    packages_to_install=[
        "google-cloud-aiplatform==1.3.0",
        "numpy==1.21.1",
        "pandas==1.3.0",
        "scikit-learn==0.24.2",
        "pyarrow==5.0.0",
        "fsspec==2021.9.0",
        "gcsfs==2021.9.0",
    ],
)
def evaluate_batch_predictions(
    batch_prediction_job: Input[Artifact],
    gcs_ground_truth: str,
    model_framework: str,
    model_type: str,
    metrics: Output[Metrics],
    reference_metric_name: str = "rmse",
) -> NamedTuple("ModelEvaluationOutput", [("metric", float)]):
    import json
    from collections import namedtuple

    import gcsfs
    import numpy as np
    import pandas as pd
    from google.cloud import aiplatform
    from sklearn.metrics import mean_squared_error

    def treat_job_uri(uri):
        return uri[uri.find("projects/") :]

    def get_job_output_dir(batch_prediction_uri):
        bpj = aiplatform.BatchPredictionJob(batch_prediction_uri)
        output = bpj.output_info.gcs_output_directory
        return output

    def create_dicts_from_predictions(local_files):
        instances_output = []
        predictions_output = []
        for x in local_files:
            with gcs.open(x) as f:
                for line in f:
                    line_json = json.loads(line)
                    # print(json.dumps(line_json, indent=2))
                    instance = line_json["instance"]
                    instances_output.append(instance)
                    pred = line_json["prediction"]
                    if type(pred) is dict and "value" in pred.keys():
                        # AutoML predictions
                        prediction = pred["value"]
                    elif type(pred) is list:
                        # BQML Predictions return different format
                        prediction = pred[0]

                    predictions_output.append(prediction)
        return instances_output, predictions_output

    def evaluate(predictions, test_results):
        return dict(rmse=np.sqrt(mean_squared_error(test_results, predictions)))

    def get_prediction_file_names(gcs, gcs_dir):
        reg_expression = f"{gcs_dir}/prediction.results*"
        file_names = [f"gs://{x}" for x in gcs.glob(reg_expression)]
        return file_names

    bpj_uri = batch_prediction_job.uri
    print(f"Batch Prediction Job URI: {bpj_uri}")

    treated_uri = treat_job_uri(bpj_uri)
    gcs_dir = get_job_output_dir(treated_uri)
    print(f"Results saved to {gcs_dir}")
    gcs = gcsfs.GCSFileSystem()
    prediction_files = get_prediction_file_names(gcs, gcs_dir)
    print(f"Predictions available on following files: {prediction_files}")

    instances, predictions = create_dicts_from_predictions(prediction_files)
    print(f"{len(predictions)} predictions found")

    labels = pd.read_csv(gcs_ground_truth).to_numpy()
    print(f"{len(labels)} ground truth labels found")

    evaluation_metric = evaluate(predictions, labels)
    print(evaluation_metric)

    print("Logging metrics to output artifact")
    metrics.metadata["model_type"] = model_type
    metrics.metadata["framework"] = model_framework
    metrics.metadata["SAMPLE_KEY"] = "You can add other metrics here"

    for k, v in evaluation_metric.items():
        print(f"{k} -> {v}")
        metrics.log_metric(k, v)
    ModelEvaluationOutput = namedtuple("ModelEvaluationOutput", ["metric"])

    return ModelEvaluationOutput(metric=float(evaluation_metric[reference_metric_name]))

### Model Selection

Now that we have evaluated the models independently, we are going to move forward with only one of them. This election will be done based on the model evaluation metrics gathered in the previous step.

In [ ]:
@component(base_image="python:3.9")
def select_best_model(
    model_one: Input[Model],
    metrics_one: float,
    model_two: Input[Model],
    metrics_two: float,
    thresholds_dict_str: str,
    best_model: Output[Model],
    reference_metric_name: str = "rmse",
) -> NamedTuple(
    "Outputs", [("deploy_decision", str), ("metric", float), ("metric_name", str)]
):
    import json
    from collections import namedtuple

    message = None
    best_metric = float("inf")
    if metrics_one <= metrics_two:
        best_model.uri = model_one.uri
        best_metric = metrics_one
        message = "Model one is the best"
    else:
        best_model.uri = model_two.uri
        best_metric = metrics_two
        message = "Model two is the best"
    thresholds_dict = json.loads(thresholds_dict_str)
    deploy = False
    if best_metric < thresholds_dict[reference_metric_name]:
        deploy = True
    if deploy:
        deploy_decision = "true"
    else:
        deploy_decision = "false"

    print(f"Which model is best? {message}")
    print(f"What metric is being used? {reference_metric_name}")
    print(f"What is the best metric? {best_metric}")
    print(f"What is the threshold to deploy? {thresholds_dict_str}")
    print(f"Deploy decision: {deploy_decision}")

    Outputs = namedtuple("Outputs", ["deploy_decision", "metric", "metric_name"])

    return Outputs(
        deploy_decision=deploy_decision,
        metric=best_metric,
        metric_name=reference_metric_name,
    )

### Validate Infrastructure

Now that the best model has been deployed, we will validate the endpoint by making a simple prediction to it.

In [ ]:
@component(base_image="python:3.9", packages_to_install=["google-cloud-aiplatform"])
def validate_infra(
    endpoint: Input[Artifact],
) -> NamedTuple(
    "validate_infrastructure_output", [("instance", str), ("prediction", float)]
):
    import json
    from collections import namedtuple

    from google.cloud import aiplatform
    from google.protobuf import json_format
    from google.protobuf.struct_pb2 import Value

    def treat_uri(uri):
        return uri[uri.find("projects/") :]

    def request_prediction(endp, instance):
        instance = json_format.ParseDict(instance, Value())
        instances = [instance]
        parameters_dict = {}
        parameters = json_format.ParseDict(parameters_dict, Value())
        response = endp.predict(instances=instances, parameters=parameters)
        print("deployed_model_id:", response.deployed_model_id)
        print("predictions: ", response.predictions)
        # The predictions are a google.protobuf.Value representation of the model's predictions.
        predictions = response.predictions

        for pred in predictions:
            if type(pred) is dict and "value" in pred.keys():
                # AutoML predictions
                prediction = pred["value"]
            elif type(pred) is list:
                # BQML Predictions return different format
                prediction = pred[0]
            return prediction

    endpoint_uri = endpoint.uri
    treated_uri = treat_uri(endpoint_uri)

    instance = {
        "Sex": "M",
        "Length": 0.33,
        "Diameter": 0.255,
        "Height": 0.08,
        "Whole_weight": 0.205,
        "Shucked_weight": 0.0895,
        "Viscera_weight": 0.0395,
        "Shell_weight": 0.055,
    }
    instance_json = json.dumps(instance)
    print("Will use the following instance: " + instance_json)

    endpoint = aiplatform.Endpoint(treated_uri)
    prediction = request_prediction(endpoint, instance)
    result_tuple = namedtuple(
        "validate_infrastructure_output", ["instance", "prediction"]
    )

    return result_tuple(instance=str(instance_json), prediction=float(prediction))

## The Pipeline

In [ ]:
DISPLAY_NAME = "rapid-prototyping-bqml-vs-automl"

pipeline_params = {
    "project": PROJECT_ID,
    "region": REGION,
    "gcs_input_file_uri": RAW_INPUT_DATA,
    "bq_dataset": BQ_DATASET,
    "bq_location": BQ_LOCATION,
    "bqml_model_export_location": BQML_EXPORT_LOCATION,
    "bqml_serving_container_image_uri": BQML_SERVING_CONTAINER_IMAGE_URI,
    "test_dataset_folder": TEST_DATASET_CSV_LOCATION,
    "gcs_batch_prediction_output_prefix": GCS_BATCH_PREDICTION_OUTPUT_PREFIX,
    "thresholds_dict_str": '{"rmse": 20.0}',
}

In [ ]:
@dsl.pipeline(name=DISPLAY_NAME, description="Rapid Prototyping")
def train_pipeline(
    project: str,
    gcs_input_file_uri: str,
    region: str,
    bq_dataset: str,
    bq_location: str,
    bqml_model_export_location: str,
    bqml_serving_container_image_uri: str,
    test_dataset_folder: str,
    gcs_batch_prediction_output_prefix: str,
    thresholds_dict_str: str,
):
    import_data_to_bigquery_op = import_data_to_bigquery(
        project, bq_location, bq_dataset, gcs_input_file_uri
    )

    job_display_name = f"{DISPLAY_NAME}_job"

    raw_dataset = import_data_to_bigquery_op.outputs["raw_dataset"]

    split_datasets_op = split_datasets(
        raw_dataset, test_dataset_folder=test_dataset_folder, bq_location=bq_location
    )

    bqml_dataset = split_datasets_op.outputs["bqml_dataset"]
    train_bqml_model_op = train_bqml_model(bqml_dataset, bq_location=bq_location)
    bqml_trained_model = train_bqml_model_op.outputs["bqml_model"]
    export_bqml_to_tf_op = export_bqml_to_tf(
        export_location=bqml_model_export_location,
        project=project,
        bqml_model=bqml_trained_model,
        bq_location=bq_location,
    )

    batch_prediction_input = split_datasets_op.outputs["test_features_jsonl_filenames"]
    ground_truth = split_datasets_op.outputs["test_labels_csv_filename"]

    bqml_model_upload_op = vertex_pipeline_components.ModelUploadOp(
        project=project,
        location=region,
        display_name=DISPLAY_NAME + "_bqml",
        artifact_uri=bqml_model_export_location,
        serving_container_image_uri=bqml_serving_container_image_uri,
    )
    bqml_model_upload_op.after(export_bqml_to_tf_op)
    bqml_model = bqml_model_upload_op.outputs["model"]

    bqml_model_batch_prediction_task = vertex_pipeline_components.ModelBatchPredictOp(
        project=project,
        location=region,
        model=bqml_model,
        job_display_name=job_display_name + "_bqml",
        gcs_source=batch_prediction_input,
        gcs_destination_prefix=gcs_batch_prediction_output_prefix,
        predictions_format="jsonl",
        instances_format="jsonl",
        machine_type="n1-standard-2",
    )

    dataset_create_op = vertex_pipeline_components.TabularDatasetCreateOp(
        project=project,
        location=region,
        display_name=DISPLAY_NAME,
        bq_source=split_datasets_op.outputs["dataset_uri"],
    )

    automl_training_op = vertex_pipeline_components.AutoMLTabularTrainingJobRunOp(
        project=project,
        location=region,
        display_name=f"{DISPLAY_NAME}_automl",
        optimization_prediction_type="regression",
        optimization_objective="minimize-rmse",
        predefined_split_column_name="split_col",
        dataset=dataset_create_op.outputs["dataset"],
        target_column="Rings",
        column_transformations=[
            {"categorical": {"column_name": "Sex"}},
            {"numeric": {"column_name": "Length"}},
            {"numeric": {"column_name": "Diameter"}},
            {"numeric": {"column_name": "Height"}},
            {"numeric": {"column_name": "Whole_weight"}},
            {"numeric": {"column_name": "Shucked_weight"}},
            {"numeric": {"column_name": "Viscera_weight"}},
            {"numeric": {"column_name": "Shell_weight"}},
            {"numeric": {"column_name": "Rings"}},
        ],
    )
    automl_model = automl_training_op.outputs["model"]
    automl_model_batch_prediction_task = vertex_pipeline_components.ModelBatchPredictOp(
        project=project,
        location=region,
        model=automl_model,
        job_display_name=job_display_name + "_automl",
        gcs_source=batch_prediction_input,
        gcs_destination_prefix=gcs_batch_prediction_output_prefix,
        predictions_format="jsonl",
        instances_format="jsonl",
        machine_type="n1-standard-2",
    )

    automl_model_evaluation_task = evaluate_batch_predictions(
        batch_prediction_job=automl_model_batch_prediction_task.outputs[
            "batchpredictionjob"
        ],
        gcs_ground_truth=ground_truth,
        model_framework="AutoML",
        model_type="Regression",
    )

    bqml_model_evaluation_task = evaluate_batch_predictions(
        batch_prediction_job=bqml_model_batch_prediction_task.outputs[
            "batchpredictionjob"
        ],
        gcs_ground_truth=ground_truth,
        model_framework="BQML",
        model_type="Regression",
    )

    automl_model_metric = automl_model_evaluation_task.outputs["metric"]
    bqml_model_metric = bqml_model_evaluation_task.outputs["metric"]

    best_model_task = select_best_model(
        model_one=automl_model,
        metrics_one=automl_model_metric,  # ,
        model_two=bqml_model,
        metrics_two=bqml_model_metric,  # automl_model_evaluation_task.outputs['metric'],
        thresholds_dict_str=thresholds_dict_str,
    )

    with dsl.Condition(
        best_model_task.outputs["deploy_decision"] == "true",
        name="deploy_decision",
    ):
        endpoint_create_op = vertex_pipeline_components.EndpointCreateOp(
            project=project,
            location=region,
            display_name=f"{DISPLAY_NAME}_endpoint",
        )

        endpoint_create_op.after(best_model_task)

        model_deploy_op = vertex_pipeline_components.ModelDeployOp(  # noqa: F841
            project=project,
            location=region,
            endpoint=endpoint_create_op.outputs["endpoint"],
            model=best_model_task.outputs["best_model"],
            deployed_model_display_name=DISPLAY_NAME + "_best",
            machine_type="n1-standard-2",
            # traffic_percentage=100
        ).set_caching_options(False)

        validate_infra_task = validate_infra(
            endpoint=model_deploy_op.outputs["endpoint"]
        ).set_caching_options(False)

        validate_infra_task.after(model_deploy_op)

### Running the Pipeline

In [ ]:
compiler.Compiler().compile(
    pipeline_func=train_pipeline,
    package_path=PIPELINE_JSON_PKG_PATH,
)

vertex.init(project=PROJECT_ID, location=REGION)

pipeline_job = vertex.PipelineJob(
    display_name=DISPLAY_NAME,
    template_path=PIPELINE_JSON_PKG_PATH,
    pipeline_root=PIPELINE_ROOT,
    parameter_values=pipeline_params,
    enable_caching=True,
)

response = pipeline_job.run()

print(response)

## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

In [ ]:
vertex.init(project=PROJECT_ID, location=REGION)

delete = False  # set to True if you want to delete resources


delete_vertex_dataset = True and delete
delete_pipeline = True and delete
delete_model = True and delete
delete_endpoint = True and delete
delete_batchjob = True and delete
delete_bucket = True and delete
delete_bq_dataset = True and delete

try:
    if delete_endpoint and "DISPLAY_NAME" in globals():
        print("Will delete endpoint")
        endpoints = vertex.Endpoint.list(
            filter=f"display_name={DISPLAY_NAME}_endpoint", order_by="create_time"
        )
        endpoint = endpoints[0]
        endpoint.undeploy_all()
        vertex.Endpoint.delete(endpoint.resource_name)
        print("Deleted endpoint:", endpoint)
except Exception as e:
    print(e)

if delete_model and "DISPLAY_NAME" in globals():
    print("Will delete models")
    suffix_list = ["bqml", "automl", "best"]
    for suffix in suffix_list:
        try:
            model_display_name = f"{DISPLAY_NAME}_{suffix}"
            print("Will delete model with name " + model_display_name)
            models = vertex.Model.list(
                filter=f"display_name={model_display_name}", order_by="create_time"
            )

            model = models[0]
            vertex.Model.delete(model)
            print("Deleted model:", model)
        except Exception as e:
            print(e)

if delete_vertex_dataset and "DISPLAY_NAME" in globals():
    print("Will delete Vertex dataset")
    try:
        datasets = vertex.TabularDataset.list(
            filter=f"display_name={DISPLAY_NAME}", order_by="create_time"
        )

        dataset = datasets[0]
        vertex.TabularDataset.delete(dataset)
        print("Deleted Vertex dataset:", dataset)
    except Exception as e:
        print(e)


try:
    if delete_pipeline and "DISPLAY_NAME" in globals():
        pipelines = vertex.PipelineJob.list(
            filter=f"pipeline_name={DISPLAY_NAME}", order_by="create_time"
        )
        pipeline = pipelines[0]
        vertex.PipelineJob.delete(pipeline)
        print("Deleted pipeline:", pipeline)
except Exception as e:
    print(e)

if delete_bq_dataset and "DISPLAY_NAME" in globals():
    from google.cloud import bigquery

    try:
        # Construct a BigQuery client object.

        bq_client = bigquery.Client(project=PROJECT_ID, location=BQ_LOCATION)

        # TODO(developer): Set model_id to the ID of the model to fetch.
        dataset_id = f"{PROJECT_ID}.{BQ_DATASET}"

        print(f"Will delete BQ dataset '{dataset_id}' from location {BQ_LOCATION}.")
        # Use the delete_contents parameter to delete a dataset and its contents.
        # Use the not_found_ok parameter to not receive an error if the dataset has already been deleted.
        bq_client.delete_dataset(
            dataset_id, delete_contents=True, not_found_ok=True
        )  # Make an API request.

        print(f"Deleted BQ dataset '{dataset_id}' from location {BQ_LOCATION}.")
    except Exception as e:
        print(e)


if delete_bucket and "BUCKET_NAME" in globals():
    ! gsutil rm -r gs://$BUCKET_NAME